# 文本向量化

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

text =["小贝来到北京清华大学", 
        "小花来到了网易杭研大厦",   
        "小明硕士毕业于中国科学院",
        "小明爱北京小明爱北京天安门"
]

# 本文不管如何检测和断句，                
corpus=["小贝 来到 北京 清华大学", 
        "小花 来到 了 网易 杭研 大厦",   
        "小明 硕士 毕业 于 中国 科学院",
        "小明 爱 北京 小明 爱 北京 天安门"
]

## 二值化、词频

### 二值化编码

In [2]:

# CountVectorizer是scikit-learn库中的一个文本特征提取方法，它将文本数据转换成词频矩阵。
# min_df=1表示忽略那些在所有文档中出现次数小于等于1的词语，即考虑所有出现过的词语。
# binary=True表示将词频矩阵中的非零值设置为1，即只考虑是否出现而不考虑出现(不管出现多少次)的次数。
vectorizer = CountVectorizer(min_df=1,binary=True) #Transformer


### 词频

In [3]:

# fit_transform方法将输入的文本数据corpus转换成词频矩阵。corpus是一个包含文档的列表或数组，每个文档可以是一段文本。
# fit_transform方法会学习词典（词汇表）并返回文本数据的词频矩阵。
data = vectorizer.fit_transform(corpus)

# get_feature_names_out方法用于获取生成的词汇表中的特征名（词语）。这里将获取的特征名存储在features变量中。
features = vectorizer.get_feature_names_out()

for word in features:
    print(word)
print(f"total {len(features)} features！")

中国
北京
大厦
天安门
小明
小花
小贝
来到
杭研
毕业
清华大学
硕士
科学院
网易
total 14 features！


In [4]:
data

<4x14 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [5]:
data.todense()

matrix([[0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
        [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [6]:
# 将CountVectorizer生成的词频矩阵（存储在data中）转换成一个DataFrame对象，并设置行索引和列名：
# pd.DataFrame是Pandas库中用于创建数据框（DataFrame）的类。
# data.toarray()将CountVectorizer生成的稀疏矩阵（在上一段代码中生成的data）转换成密集矩阵，即将文本数据的词频矩阵转换为二维数组。
# index=text设置数据框的行索引为text。
# columns=vectorizer.get_feature_names_out()设置数据框的列名为词汇表中的词语。get_feature_names_out()是CountVectorizer的方法，用于获取生成的词汇表中的特征名（词语）。

doc_df = pd.DataFrame(data.toarray(), index=text, columns=vectorizer.get_feature_names_out()).head(10)

In [7]:
doc_df

,中国,北京,大厦,天安门,小明,小花,小贝,来到,杭研,毕业,清华大学,硕士,科学院,网易
小贝来到北京清华大学,0,1,0,0,0,0,1,1,0,0,1,0,0,0
小花来到了网易杭研大厦,0,0,1,0,0,1,0,1,1,0,0,0,0,1
小明硕士毕业于中国科学院,1,0,0,0,1,0,0,0,0,1,0,1,1,0
小明爱北京小明爱北京天安门,0,1,0,1,1,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sims = cosine_similarity(doc_df)

In [9]:
cos_sims

array([[1.        , 0.2236068 , 0.        , 0.28867513],
       [0.2236068 , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.25819889],
       [0.28867513, 0.        , 0.25819889, 1.        ]])

In [10]:
sims_df = pd.DataFrame(cos_sims, index = text, columns = text)

In [11]:
sims_df

,小贝来到北京清华大学,小花来到了网易杭研大厦,小明硕士毕业于中国科学院,小明爱北京小明爱北京天安门
小贝来到北京清华大学,1.000000,0.223607,0.000000,0.288675
小花来到了网易杭研大厦,0.223607,1.000000,0.000000,0.000000
小明硕士毕业于中国科学院,0.000000,0.000000,1.000000,0.258199
小明爱北京小明爱北京天安门,0.288675,0.000000,0.258199,1.000000


## tf-idf
TF - IDF 法在词频法的基础上进行了改进。它对每个单词出现的次数做了修正，对于那些常见词，比如"你""我""他""是"这样一些毫无区分度、在所有文章中都会大量出现的单词降低了它的频次，从而减少这个维度的重要性。
而对于一些非常罕见、有非常强的区分能力的单词， TF - IDF 会调高它的频次。比如"信息熵"，这是个非常冷门的计算机术语，只会出现在一些专业论文里，不出现则已，一出现则锋芒毕现。像这样的词，要提升它的重要性。
上面4句话经过 TF - IDF 编码后，得到的结果：

In [12]:
vectorizer = TfidfVectorizer(min_df=1)
data = vectorizer.fit_transform(corpus)
features = vectorizer.get_feature_names_out()
for word in features:
    print(word)

中国
北京
大厦
天安门
小明
小花
小贝
来到
杭研
毕业
清华大学
硕士
科学院
网易


In [13]:
pd.set_option('display.precision', 2)
doc_df = pd.DataFrame(data.toarray(), index=text, columns=vectorizer.get_feature_names_out()).head(10)
doc_df

,中国,北京,大厦,天安门,小明,小花,小贝,来到,杭研,毕业,清华大学,硕士,科学院,网易
小贝来到北京清华大学,0.00,0.44,0.00,0.00,0.00,0.00,0.56,0.44,0.00,0.00,0.56,0.00,0.00,0.00
小花来到了网易杭研大厦,0.00,0.00,0.47,0.00,0.00,0.47,0.00,0.37,0.47,0.00,0.00,0.00,0.00,0.47
小明硕士毕业于中国科学院,0.47,0.00,0.00,0.00,0.37,0.00,0.00,0.00,0.00,0.47,0.00,0.47,0.47,0.00
小明爱北京小明爱北京天安门,0.00,0.65,0.00,0.41,0.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
